## full version, counting and estimate frequency for all arriving elements

In [19]:
# precision 約69~70%, top-256, cms:8*1034, execution time: 76xxx秒


import sys
import os
import re
import time
from probables import (CountMinSketch)
import pandas as pd

def find(e,Topk):
    try:
        index = [ele for ele,i in Topk].index(e)
        return index
    except:
        index=-99
    return index
#==================== main =============================
Top=[]
w=1024
d=8
size=256
# item_count=100
cms = CountMinSketch(width=w, depth=d)

#datapath='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
datapath='..\dataset\webdocs'
# path=r'C:\Users\Pero\python\Python\webdocs'
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(datapath)))

start=time.time()
for datafile in filelist:
    src_data=os.path.join(datapath,datafile)
    if os.path.exists(src_data):
        with open(src_data,'r') as file:
            while True:
                e=file.readline().strip('\n')
                if not e:
                    print('EOF')
                    break
                else:
                    #item_count-=1
                    #print("read {}th element: {}".format(item_count,line[:10]))
                    cms.add(e)
                    count=cms.check(e)
                    index=find(e,Top)
                    if index >=0:
                        Top[index][1]=count
                    else:
                        # e not in Top
                        if len(Top)<size:
                            # Top is not full
                            Top.append([e,count])
                            index=len(Top)-1
                            #print("id of iindex:{}".format(id(index)))
                        else:
                            # Top if full
                            if count< Top[-1][1]:
                                pass
                            else:
                                Top.append([e,count])
                                Top=sorted(Top,key = lambda Top:Top[1],reverse=True)
                                Top.pop()
                                #rint("Top after pop:{}\n".format(Top))
                    Top=sorted(Top,key = lambda Top:Top[1],reverse=True)
        # print(topk[:20],len(topk))
    else:
        print("file doesn't exist")
        
end=time.time()
print("Total memory {3} bytes :Top-{0} with size {1} bytes+ CMS with size {2} bytes.".format(len(Top),sys.getsizeof(Top),sys.getsizeof(cms._bins),sys.getsizeof(cms._bins)+sys.getsizeof(Top)))
print("Execution time:{:8.3f} seconds.".format(end-start))

#====================Top to csv=============================
templi=[[i[0],i[1]] for i in Top]
df=pd.DataFrame(templi,columns=['ID', 'Count'])
name="CMS_webdocs_Top-"+str(size)+'_'+str(d)+'_'+str(w)+'.csv'
df.to_csv(os.path.join('result',name),index=False)

# ====================precision====================
#gr_path='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
gr_truth='..\dataset\webdocs'
gr_file_name='webdocs_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(gr_truth,gr_file_name))
    # ground truth of webdocs
CMS_result=pd.read_csv(os.path.join('result',name))
    # result of Top-k elements and estimated frequencies

# precision
gt_set=set(grtruth['Element'][:size])
cms_set=set(CMS_result['ID'])
precision=len(gt_set & cms_set)/len(cms_set)
    # &: set 交集運算
    # precision= tp/tp+fp=tp/size
print("Precision: {}".format(precision))

# ====================ARE/AAE for Top and all====================
gt_dict=dict(grtruth.values.tolist())
top_are=0
top_aae=0
all_are=0
all_aae=0
tp=0
fp=0

read=0
startx=time.time()
for item in grtruth['Element']:
    count=cms.check(str(item))
    if read<size:
        top_are+=abs(count-gt_dict[item])/gt_dict[item]
        top_aae+=abs(count-gt_dict[item])
        read+=1
        if item in cms_set:
            tp+=1
        else:
            fp+=1
    all_are+=abs(count-gt_dict[item])/gt_dict[item]
    all_aae+=abs(count-gt_dict[item])
endx=time.time()

distinct=len(gt_dict)
top_are/=size
all_are/=distinct
all_aae/=distinct 
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.3f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.3f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

# ====================Cover result into a dataframe====================
sketch_size=str(d)+'*'+str(w)
temp=sys.getsizeof(Top)+sys.getsizeof(cms._bins)
memory_usage=str(temp)+' bytes ='+'Top:'+str(sys.getsizeof(Top))+'+ Sketch:'+str(sys.getsizeof(cms._bins))

result_df=pd.DataFrame(columns=['Top-k',
                                'Sketch',
                                'Total memory',
                                'Exe_time',
                                'Find',
                                'Precision',
                                'ARE-Top',
                                'AAE-Top',
                                'ARE-all',
                                'AAE-all'])
output_dict={
    'Top-k':size,
    'Sketch':sketch_size,
    'Total memory':memory_usage,
    'Exe_time':float('{:8.3f}'.format(end-start)),
    'Find':"Find:{}, TP:{}, FP:{}".format(len(gt_set & cms_set),tp,fp),
    'Precision':float("{:8.4f}".format(precision)),
    'ARE-Top':float('{:8.6f}'.format(top_are)),
    'AAE-Top':float('{:8.6f}'.format(top_aae)),
    'ARE-all':float('{:8.6f}'.format(all_ARE)),
    'AAE-all':float('{:8.6f}'.format(all_AAE))}

result_df=result_df.append(output_dict,ignore_index=True)
file="CMS_webdocs_result"+'_'+str(size)+'_'+str(d)+'_'+str(w)+'_.csv'
    # 程式執行結果供複製用
result_df.to_csv(os.path.join(os.path.join(r'..\result',file)),index=False)
result_df


KeyboardInterrupt: 

## count and estimate after all

In [ ]:

import sys
import os
import re
import time
from probables import (CountMinSketch)
import pandas as pd

def find(e,Topk):
    try:
        index = [ele for ele,i in Topk].index(e)
        return index
    except:
        index=-99
    return index
#==================== main =============================

Top=[]
w=1024
d=8
size=256
# item_count=100
cms = CountMinSketch(width=w, depth=d)

datapath='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
# path=r'C:\Users\Pero\python\Python\webdocs'
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(datapath)))

start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(datapath,datafile)
    if os.path.exists(src_data):
        with open(src_data,'r') as file:
            while True:
                e=file.readline().strip('\n')
                if not e:
                    print('EOF')
                    break
                else:
                    #item_count-=1
                    #print("read {}th element: {}".format(item_count,line[:10]))
                    cms.add(e)
                    '''
                    count=cms.check(e)
                    index=find(e,Top)
                    if index >=0:
                        Top[index][1]=count
                    else:
                        # e not in Top
                        if len(Top)<size:
                            # Top is not full
                            Top.append([e,count])
                            index=len(Top)-1
                            #print("id of iindex:{}".format(id(index)))
                        else:
                            # Top if full
                            if count< Top[-1][1]:
                                pass
                            else:
                                Top.append([e,count])
                                Top=sorted(Top,key = lambda Top:Top[1],reverse=True)
                                Top.pop()
                                #rint("Top after pop:{}\n".format(Top))
                    Top=sorted(Top,key = lambda Top:Top[1],reverse=True)                    
                    '''
        # print(topk[:20],len(topk))
    else:
        print("file doesn't exist")
end=time.time()

print("Execution time:{:8.3f} seconds.".format(end-start))  
print("Total memory {} bytes".format(sys.getsizeof(cms._bins)))

# ====================ARE/AAE for all====================

gr_path='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
gr_file_name='webdocs_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(gr_path,gr_file_name))
gt_dict=dict(grtruth.values.tolist())
top_are=0
top_aae=0
all_are=0
all_aae=0

read=0

startx=time.time()
for item in grtruth['Element']:
    count=cms.check(str(item))    
    if read<size:
        # ARE/AAE of Top
        top_are+=abs(count-gt_dict[item])/gt_dict[item]
        top_aae+=abs(count-gt_dict[item])
        read+=1
    # ARE/AAE of others
    all_are+=abs(count-gt_dict[item])/gt_dict[item]
    all_aae+=abs(count-gt_dict[item])
endx=time.time()

distinct=len(gt_dict)
top_are/=size
all_are/=distinct
all_aae/=distinct 
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.3f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.3f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))